## 感知机  [nn.Linear()](https://www.cnblogs.com/wupiao/articles/13288616.html)

In [1]:
import torch
x = torch.randn(10, 3)  # 输入的维度是（128，20）
m = torch.nn.Linear(3, 4)  # 20,30是指维度
output = m(x)
print('m.weight.shape:\n ', m.weight.shape)
print('m.bias.shape:\n', m.bias.shape)
print('output.shape:\n', output.shape)

# ans = torch.mm(input,torch.t(m.weight))+m.bias 等价于下面的
ans = torch.mm(x, m.weight.t()) + m.bias #torch.mm(a, b)是矩阵a和b矩阵相乘
print('ans.shape:\n', ans.shape)

print(torch.equal(ans, output))


m.weight.shape:
  torch.Size([4, 3])
m.bias.shape:
 torch.Size([4])
output.shape:
 torch.Size([10, 4])
ans.shape:
 torch.Size([10, 4])
True


In [24]:
print(m.weight)
print(m.bias)

Parameter containing:
tensor([[-0.5675,  0.3779,  0.1213],
        [ 0.1757,  0.3552,  0.5695],
        [ 0.1730, -0.4120, -0.3574],
        [-0.0985, -0.4828, -0.3555]], requires_grad=True)
Parameter containing:
tensor([ 0.3839,  0.0083,  0.4317, -0.2801], requires_grad=True)


In [31]:
c1=torch.mm(x,m.weight.t())+m.bias

In [29]:
c2=m(x)

In [49]:
import numpy as np

a1=[1,2,3,4]
a2=np.array(a1)
print(type(a1))
print(type(a2))

<class 'list'>
<class 'numpy.ndarray'>


*Tensor在cpu上才能转numpy* 
```
for name, para in model.named_parameters():
    if "ssf_scale" in name:
        ssf_scale[name]=para.data.detach().cpu().numpy()
    elif "ssf_shift" in name:
        ssf_shift[name]=para.data.detach().cpu().numpy()
    else:
        pass
```

In [43]:
bb= (c1 == c2)
bb.flatten().numpy()

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True])

In [19]:
x = torch.randn(10, 3)
print(x.shape)
print(x.t().shape)
print(x.shape)
x.t()

torch.Size([10, 3])
torch.Size([3, 10])
torch.Size([10, 3])


tensor([[ 0.1982,  1.5363,  0.0549, -0.9635, -1.2077, -0.1312,  0.3601,  0.2268,
          0.0562, -0.4309],
        [-0.8356,  0.3603,  0.8435, -0.5880,  0.2928,  1.7503,  1.2666, -0.2376,
         -0.4501,  1.4948],
        [-0.5694,  1.1981,  0.0665, -1.4922,  1.1955, -0.7883,  0.1208,  0.4790,
         -0.3644, -1.7003]])

## PyTorch的[Hook](https://blog.csdn.net/weixin_44878336/article/details/133859089)函数

In [15]:
import torch

def hook_fn(grad):
    grad *= 2
    print(grad)
    return grad

x = torch.Tensor([1, 2]).requires_grad_()
y = torch.Tensor([3, 4]).requires_grad_()
z = ((y-x) ** 2).mean()

z.register_hook(hook_fn)
# z.register_hook(lambda x: 2*x)

print("backward前")
z.backward()
print("backward后\n")

print(f"x.requires_grad: {x.requires_grad}")
print(f"y.requires_grad: {y.requires_grad}")
print(f"z.requires_grad: {z.requires_grad}\n")

print(f"x.grad: {x.grad}")
print(f"y.grad: {y.grad}")
print(f"z.grad: {z.grad}")

backward前
tensor(2.)
backward后

x.requires_grad: True
y.requires_grad: True
z.requires_grad: True

x.grad: tensor([-4., -4.])
y.grad: tensor([4., 4.])
z.grad: None


In [8]:
z=((y-x)**2).mean()
print(z.backward)

<bound method Tensor.backward of tensor(4., grad_fn=<MeanBackward0>)>


## Positional_encoding

In [ ]:
'''
代码来自 : https://github.com/wzlxjtu/PositionalEncoding2D/blob/master/positionalembedding2d.py
'''

import torch
import math
import matplotlib.pyplot as plt


def positional_encoding(d_model, length):
    """
    :param d_model: dimension of the token
    :param length: (maximum) token number
    :return: length*d_model position matrix
    """
    if d_model % 2 != 0:
        raise ValueError("Cannot use sin/cos positional encoding with "
                         "odd dim (got dim={:d})".format(d_model))
    pe = torch.zeros(length, d_model)
    position = torch.arange(0, length).unsqueeze(1)
    div_term = torch.exp((torch.arange(0, d_model, 2, dtype=torch.float) *
                         -(math.log(10000.0) / d_model)))
    pe[:, 0::2] = torch.sin(position.float() * div_term)
    pe[:, 1::2] = torch.cos(position.float() * div_term)

    return pe


pe = positional_encoding(128, 10)
plt.plot(range(10), pe[:, 0])
plt.show()

## [彻底理解 Pytorch 中的 squeeze() 和 unsqueeze()函数](https://zhuanlan.zhihu.com/p/368920094)

In [39]:
a=torch.arange(0,6).view(2,1,3,1)
print(a.shape)
b=a.squeeze(0)
print(a.shape)
print(b.shape)


torch.Size([2, 1, 3, 1])
torch.Size([2, 1, 3, 1])
torch.Size([2, 1, 3, 1])


## [DropPath](https://blog.csdn.net/qq_43426908/article/details/121662843)用法

In [200]:
from torch import nn
def drop_path(x, drop_prob: float = 0., training: bool = False):
    if drop_prob == 0. or not training:
        return x
    keep_prob = 1 - drop_prob
    shape = (x.shape[0],) + (1,) * (x.ndim - 1) 
    print(shape) 
    random_tensor = keep_prob + torch.rand(shape, dtype=x.dtype, device=x.device)
    print(random_tensor)
    random_tensor.floor_()  # binarize
    print(random_tensor)
    print(x.div(keep_prob),random_tensor)
    output = x.div(keep_prob) * random_tensor
    return output


# class DropPath(nn.Module):
#     def __init__(self, drop_prob=None):
#         super(DropPath, self).__init__()
#         self.drop_prob = drop_prob

#     def forward(self, x):
#         return drop_path(x, self.drop_prob, self.training)

a=torch.randn(2,5)
print(a)
print(drop_path(a,0.4,True))


tensor([[-0.4926, -0.9163, -2.8298,  1.8312, -0.5280],
        [ 0.8809,  0.2158, -0.3715, -1.8313, -0.5842]])
(2, 1)
tensor([[0.6601],
        [1.2221]])
tensor([[0.],
        [1.]])
tensor([[-0.8209, -1.5272, -4.7163,  3.0519, -0.8800],
        [ 1.4682,  0.3596, -0.6192, -3.0521, -0.9737]]) tensor([[0.],
        [1.]])
tensor([[-0.0000, -0.0000, -0.0000,  0.0000, -0.0000],
        [ 1.4682,  0.3596, -0.6192, -3.0521, -0.9737]])


In [133]:
torch.rand((1,1), dtype=x.dtype, device=x.device)

tensor([[0.1584]])

In [106]:
print((1,)+(1,)+(1,))

(1, 1, 1)


In [171]:
x=torch.arange(0,10).view(2,5)
print(x)
x.div(0.8)

tensor([[0, 1, 2, 3, 4],
        [5, 6, 7, 8, 9]])


tensor([[ 0.0000,  1.2500,  2.5000,  3.7500,  5.0000],
        [ 6.2500,  7.5000,  8.7500, 10.0000, 11.2500]])